# Load Packages

In [23]:
using CSV
using DataFrames
using Gurobi
using Random
using Distances
using JuMP
using Gurobi

const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu


# Load the data

In [ ]:
model = Model(Gurobi.Optimizer)

# Load data
full_pairs = CSV.read("./data/kidney_data_with_states.csv", DataFrame)
centers = CSV.read("./data/Transplant_Center_Dataset.csv", DataFrame)

# get the first 10000 pairs of the data
pair_size = 10000
# center_size = 6
pairs = full_pairs[1:pair_size, :]
altruistic_donors = pairs[pairs.Altruist .== 1, :]

n = nrow(pairs)
m = nrow(centers)

println("Number of pairs: ", n)
println("Number of centers: ", m)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Number of pairs: 100
Number of centers: 256


# Base Model

In [ ]:
# Create Compatibility Matrix Based on Exact Blood Type Matches
c = zeros(Int, n, n)  # Initialize n x n matrix with zeros

# i is the index of the donor-patient pair
# j is the index of the another donor-patient pair

# we should build a compatibility matrix to match the first pair's donar with the second pair's patient

# Populate the Compatibility Matrix
for i in 1:n
    donor = pairs[i, :Donor]  # Donor blood type from pair i
    for j in 1:n
        patient = pairs[j, :Patient]  # Patient blood type from pair j
        # Exact blood type match
        c[i, j] = donor == patient ? 1 : 0
    end
end

# Create JuMP Model with Gurobi Optimizer
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1) 

# Define Decision Variables: y[i,j] = 1 if donor i donates to patient j
@variable(model, y[1:n, 1:n], Bin)

# Define Objective: Maximize the Total Number of Transplants
@objective(model, Max, sum(y[i, j] for i in 1:n for j in 1:n))

# Add Compatibility Constraints: y[i,j] <= c[i,j]
@constraint(model, [i=1:n, j=1:n], y[i, j] <= c[i, j])

# Each Donor Can Donate to At Most One Patient: Σ y[i,j] <= 1, ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) <= 1)

# Each Patient Can Receive at Most One Kidney: Σ y[i,j] <= 1, ∀ j
@constraint(model, [j=1:n], sum(y[i, j] for i in 1:n) <= 1)

# Reciprocal Donation Constraints: Σ y[i,j] == Σ y[j,i], ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) == sum(y[j, i] for j in 1:n))

# Optimize the Model
optimize!(model)

# Check Optimization Status and Extract Results
status = termination_status(model)
if status == MOI.OPTIMAL
    println("\nOptimal solution found:\n")
    transplants = DataFrame(Donor = Int[], Patient = Int[])
    for i in 1:n
        for j in 1:n
            if value(y[i, j]) > 0.5
                println("Donor $(i) donates to Patient $(j)")
                push!(transplants, (Donor = i, Patient = j))
            end
        end
    end
    println("\nTotal optimal transplants: ", nrow(transplants))
    # display(transplants)
else
    println("\nNo optimal solution found. Status: ", status)
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 10300 rows, 10000 columns and 49800 nonzeros
Model fingerprint: 0x06bb70a9
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 10026 rows and 7726 columns
Presolve time: 0.03s
Presolved: 274 rows, 2274 columns, 8409 nonzeros
Varia

# Base Model with Altrustic Donors

In [ ]:
# Create Compatibility Matrix Based on Exact Blood Type Matches
c = zeros(Int, n, n)  # Initialize n x n matrix with zeros

# now we introduce the altrustic donor, we assume altrustic donor have a dummy patient that compatible with all other donors in other pairs
altruistic_donors = findall(pairs[!, :Altruist] .== 1)

n_altruistic = length(altruistic_donors)

# Initialize Compatibility Matrix: Include extra rows and columns for dummy patients
c = zeros(Int, n, n)

# Populate compatibility for original donor-patient pairs
for i in 1:n
    donor = pairs[i, :Donor]  # Donor blood type from pair i
    for j in 1:n
        patient = pairs[j, :Patient]  # Patient blood type from pair j
        # Exact blood type match
        c[i, j] = donor == patient ? 1 : 0
    end
end

# Populate compatibility for altruistic donors
for k in altruistic_donors
    # Set compatibility of dummy patient to all donors
    for i in 1:n
        c[i, k] = 1
    end
end

# Create JuMP Model with Gurobi Optimizer
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 1) 


# Define Decision Variables: y[i,j] = 1 if donor i donates to patient j
@variable(model, y[1:n, 1:n], Bin)

# Define Objective: Maximize the Total Number of Transplants
@objective(model, Max, sum(y[i, j] for i in 1:n for j in 1:n))

# Add Compatibility Constraints: y[i,j] <= c[i,j]
@constraint(model, [i=1:n, j=1:n], y[i, j] <= c[i, j])

# Each Donor Can Donate to At Most One Patient: Σ y[i,j] <= 1, ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) <= 1)

# Each Patient Can Receive at Most One Kidney: Σ y[i,j] <= 1, ∀ j
@constraint(model, [j=1:n], sum(y[i, j] for i in 1:n) <= 1)

# Reciprocal Donation Constraints: Σ y[i,j] == Σ y[j,i], ∀ i
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) == sum(y[j, i] for j in 1:n))

# Optimize the Model
optimize!(model)

# Check Optimization Status and Extract Results
status = termination_status(model)
if status == MOI.OPTIMAL
    println("\nOptimal solution found:\n")
    transplants = DataFrame(Donor = Int[], Patient = Int[])
    for i in 1:n
        for j in 1:n
            if value(y[i, j]) > 0.5
                println("Donor $(i) donates to Patient $(j)")
                push!(transplants, (Donor = i, Patient = j))
            end
        end
    end
    println("\nTotal optimal transplants: ", nrow(transplants))
    # display(transplants)
else
    println("\nNo optimal solution found. Status: ", status)
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 10300 rows, 10000 columns and 49800 nonzeros
Model fingerprint: 0x06bb70a9
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 16.0000000
Presolve removed 10026 rows and 7726 columns
Presolve time: 0.03s
Presolved: 274 rows, 2274 columns, 8409 nonzeros
Varia

# Introducing Matching Score

In [13]:
# Create a compatibility dictionary with matching scores
compatibility_scores = Dict(
    ("O", "O") => 10, ("O", "A") => 8, ("O", "B") => 8, ("O", "AB") => 5,
    ("A", "A") => 10, ("A", "AB") => 7,
    ("B", "B") => 10, ("B", "AB") => 7,
    ("AB", "AB") => 10
)

# Create a compatibility matrix with scores
s = zeros(Int, n, n)  # Initialize compatibility matrix

# Populate compatibility for original donor-patient pairs
for i in 1:n
    donor = pairs[i, :Donor]  # Donor blood type from pair i
    for j in 1:n
        patient = pairs[j, :Patient]  # Patient blood type from pair j
        # Exact blood type match
        s[i, j] = get(compatibility_scores, (donor, patient), 0)
    end
end

# Populate compatibility for altruistic donors
for k in altruistic_donors
    # Set compatibility of dummy patient to all donors
    for i in 1:n
        # assign a 10 score to the dummy patient
        s[i, k] = 10
    end
end


# Create JuMP model with Gurobi optimizer
model = Model(Gurobi.Optimizer)

# Define decision variables: y[i,j] = 1 if donor i donates to patient j
@variable(model, y[1:n, 1:n], Bin)

# Define objective: Maximize the total matching score
@objective(model, Max, sum(s[i, j] * y[i, j] for i in 1:n, j in 1:n))

# Add Compatibility Constraints: y[i,j] can be 1 only if s[i,j] > 0
@constraint(model, [i=1:n, j=1:n], y[i, j] <= (s[i, j] > 0 ? 1 : 0))

# Each donor can donate to at most one patient
@constraint(model, [i=1:n], sum(y[i, j] for j in 1:n) <= 1)

# Each patient can receive from at most one donor
@constraint(model, [j=1:n], sum(y[i, j] for i in 1:n) <= 1)

# Optimize the model
optimize!(model)

# Check Optimization Status and Extract Results
status = termination_status(model)
if status == MOI.OPTIMAL
    println("\nOptimal solution found with maximum matching score:\n")
    total_score = 0
    matches = DataFrame(Donor = Int[], Patient = Int[], Score = Int[])
    for i in 1:n
        for j in 1:n
            if value(y[i, j]) > 0.5
                score = s[i, j]
                println("Donor $(i) donates to Patient $(j) with score $(score)")
                total_score += score
                push!(matches, (Donor = i, Patient = j, Score = score))
            end
        end
    end
    println("\nTotal Matching Score: ", total_score)
    # Uncomment the following line to display the matches DataFrame
    # display(matches)
else
    println("\nNo optimal solution found. Status: ", status)
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 10200 rows, 10000 columns and 30000 nonzeros
Model fingerprint: 0x98ec0bec
Variable types: 0 continuous, 10000 integer (10000 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 510.0000000
Presolve removed 10014 rows and 7240 columns
Presolve time: 0.04s
Presolved: 186 rows, 2760 columns, 5520 nonzeros
Vari

# Introducing Transportation Cost

In [ ]:
# Calculate distances
distances = Dict()
for i in 1:n, k in 1:m
    pair_coords = (pairs[i, :lat], pairs[i, :lng])
    center_coords = (centers[k, :lat], centers[k, :lng])
    distances[(i, k)] = haversine(pair_coords, center_coords, 6371)  # 6371 is Earth's radius in km
end

# Create a compatibility dictionary with matching scores
compatibility_scores = Dict(
    ("O", "O") => 10, ("O", "A") => 8, ("O", "B") => 8, ("O", "AB") => 5,
    ("A", "A") => 10, ("A", "AB") => 7,
    ("B", "B") => 10, ("B", "AB") => 7,
    ("AB", "AB") => 10
)

# Create a compatibility matrix with scores
s = zeros(Int, n, n)  # Initialize compatibility matrix

# Populate compatibility for original donor-patient pairs
for i in 1:n
    donor = pairs[i, :Donor]  # Donor blood type from pair i
    for j in 1:n
        patient = pairs[j, :Patient]  # Patient blood type from pair j
        # Exact blood type match
        s[i, j] = get(compatibility_scores, (donor, patient), 0)
    end
end

# Populate compatibility for altruistic donors
for k in altruistic_donors
    # Set compatibility of dummy patient to all donors
    for i in 1:n
        # assign a 10 score to the dummy patient
        s[i, k] = 10
    end
end


# Create JuMP model with Gurobi optimizer
model = Model(Gurobi.Optimizer)

# Calculate d_max from the data
d_max = 0.0
for i in 1:n, j in 1:n, k in 1:m
    # Calculate total distance for this potential pairing through center k
    total_distance = distances[(i,k)] + distances[(j,k)]
    global d_max = max(d_max, total_distance)
end

α = 1.0 / (2 * d_max * n)

# Decision variables
@variable(model, y[1:n, 1:n, 1:m], Bin)  # y[i,j,k] = 1 if donor i donates to patient j through center k
@variable(model, z[1:m], Bin)  # z[k] = 1 if center k is used

# Objective function
@objective(model, Max, 
    # First term: maximize matching scores
    sum(s[i, j] * y[i,j,k] for i in 1:n for j in 1:n for k in 1:m) -
    
    # Second term: minimize travel distance weighted by α
    α * sum((distances[(i,k)] + distances[(j,k)]) * y[i,j,k] for i in 1:n for j in 1:n for k in 1:m)
)

# Add Compatibility Constraints: y[i,j] can be 1 only if s[i,j] > 0
@constraint(model, [i=1:n, j=1:n, k=1:m], y[i, j, k] <= (s[i, j] > 0 ? 1 : 0))

# Each donor can donate to at most one patient
@constraint(model, [i=1:n], sum(y[i, j, k] for j in 1:n for k in 1:m) <= 1)

# Each patient can receive from at most one donor
@constraint(model, [j=1:n], sum(y[i, j, k] for i in 1:n for k in 1:m) <= 1)

# Optimize the model
optimize!(model)

# Print results
if termination_status(model) == MOI.OPTIMAL
    println("\nOptimal Solution Found:")
    for i in 1:n, j in 1:n, k in 1:m
        if value(y[i,j,k]) > 0.5
            println("Donor $i donates to Patient $j through Center $k")
        end
    end
    
    used_centers = [k for k in 1:m if value(z[k]) > 0.5]
    println("\nSelected Transplant Centers: ", [centers[k, Symbol("mix-text_weightBold")] for k in used_centers])
    
    total_exchanges = sum(value(y[i,j,k]) for i in 1:n for j in 1:n for k in 1:m)
    total_travel_distance = sum((distances[(i,k)] + distances[(j,k)]) * value(y[i,j,k]) 
                               for i in 1:n for j in 1:n for k in 1:m)
    
    println("\nTotal Exchanges: ", total_exchanges)
    println("Total Travel Distance: ", round(total_travel_distance, digits=2), " km")
    println("Objective Value: ", round(objective_value(model), digits=2))
else
    println("No optimal solution found.")
end

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2553220
Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 9 7940HS w/ Radeon 780M Graphics, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2553220 - for non-commercial use only - registered to ga___@mit.edu
Optimize a model with 2560712 rows, 2560256 columns and 12800449 nonzeros
Model fingerprint: 0xecaa9b88
Variable types: 0 continuous, 2560256 integer (2560256 binary)
Coefficient statistics:
  Matrix range     [1e-04, 2e+02]
  Objective range  [2e-08, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Found heuristic solution: objective 509.9369467
Presolve removed 2560319 rows and 2008276 columns
Presolve ti

# Introducing Surgery Constrains

In [ ]:
# Calculate distances
distances = Dict()
for i in 1:n, k in 1:m
    pair_coords = (pairs[i, :lat], pairs[i, :lng])
    center_coords = (centers[k, :lat], centers[k, :lng])
    distances[(i, k)] = haversine(pair_coords, center_coords, 6371)  # 6371 is Earth's radius in km
end

# Create a compatibility dictionary with matching scores
compatibility_scores = Dict(
    ("O", "O") => 10, ("O", "A") => 8, ("O", "B") => 8, ("O", "AB") => 5,
    ("A", "A") => 10, ("A", "AB") => 7,
    ("B", "B") => 10, ("B", "AB") => 7,
    ("AB", "AB") => 10
)

# Create a compatibility matrix with scores
s = zeros(Int, n, n)  # Initialize compatibility matrix

# Populate compatibility for original donor-patient pairs
for i in 1:n
    donor = pairs[i, :Donor]  # Donor blood type from pair i
    for j in 1:n
        patient = pairs[j, :Patient]  # Patient blood type from pair j
        # Exact blood type match
        s[i, j] = get(compatibility_scores, (donor, patient), 0)
    end
end

# Populate compatibility for altruistic donors
for k in altruistic_donors
    # Set compatibility of dummy patient to all donors
    for i in 1:n
        # assign a 10 score to the dummy patient
        s[i, k] = 10
    end
end


# Create JuMP model with Gurobi optimizer
model = Model(Gurobi.Optimizer)

# Calculate d_max from the data
d_max = 0.0
for i in 1:n, j in 1:n, k in 1:m
    # Calculate total distance for this potential pairing through center k
    total_distance = distances[(i,k)] + distances[(j,k)]
    global d_max = max(d_max, total_distance)
end

α = 1.0 / (2 * d_max * n)

# Decision variables
@variable(model, y[1:n, 1:n, 1:m], Bin)  # y[i,j,k] = 1 if donor i donates to patient j through center k
@variable(model, z[1:m], Bin)  # z[k] = 1 if center k is used

# Objective function
@objective(model, Max, 
    # First term: maximize matching scores
    sum(s[i, j] * y[i,j,k] for i in 1:n for j in 1:n for k in 1:m) -
    
    # Second term: minimize travel distance weighted by α
    α * sum((distances[(i,k)] + distances[(j,k)]) * y[i,j,k] for i in 1:n for j in 1:n for k in 1:m)
)

# Add Compatibility Constraints: y[i,j] can be 1 only if s[i,j] > 0
@constraint(model, [i=1:n, j=1:n, k=1:m], y[i, j, k] <= (s[i, j] > 0 ? 1 : 0))

# Each donor can donate to at most one patient
@constraint(model, [i=1:n], sum(y[i, j, k] for j in 1:n for k in 1:m) <= 1)

# Each patient can receive from at most one donor
@constraint(model, [j=1:n], sum(y[i, j, k] for i in 1:n for k in 1:m) <= 1)

# Transplant center capacity limitation
for k in 1:m
    capacity = centers[k, Symbol("Living Donor Transplants in a year")]
    @constraint(model, sum(y[i,j,k] for i in 1:n for j in 1:n) <= capacity * z[k])
end

# Center usage constraint limitation
for k in 1:m
    @constraint(model, z[k] >= sum(y[i,j,k] for i in 1:n for j in 1:n) / (n * n))
end

# Optimize the model
optimize!(model)

# Print results
if termination_status(model) == MOI.OPTIMAL
    println("\nOptimal Solution Found:")
    for i in 1:n, j in 1:n, k in 1:m
        if value(y[i,j,k]) > 0.5
            println("Donor $i donates to Patient $j through Center $k")
        end
    end
    
    used_centers = [k for k in 1:m if value(z[k]) > 0.5]
    println("\nSelected Transplant Centers: ", [centers[k, Symbol("mix-text_weightBold")] for k in used_centers])
    
    total_exchanges = sum(value(y[i,j,k]) for i in 1:n for j in 1:n for k in 1:m)
    total_travel_distance = sum((distances[(i,k)] + distances[(j,k)]) * value(y[i,j,k]) 
                               for i in 1:n for j in 1:n for k in 1:m)
    
    println("\nTotal Exchanges: ", total_exchanges)
    println("Total Travel Distance: ", round(total_travel_distance, digits=2), " km")
    println("Objective Value: ", round(objective_value(model), digits=2))
else
    println("No optimal solution found.")
end